In [1]:
import pandas as pd

In [2]:
import json

In [3]:
pd.set_option('display.max_rows', 500)

In [12]:
with open('data/world_bank_projects.json') as json_file:
    data = json.load(json_file)
    
    # get the data - read into pandas df
df = pd.read_json('data/world_bank_projects.json')
print (df.shape)
df.describe()


(500, 50)


,idacommamt,totalcommamt,totalamt,approvalfy,lendprojectcost,grantamt,ibrdcommamt
count,5.000000e+02,5.000000e+02,5.000000e+02,500.000000,5.000000e+02,5.000000e+02,5.000000e+02
mean,3.542136e+07,7.271386e+07,6.828146e+07,2013.108000,1.547241e+08,4.432400e+06,3.286010e+07
std,7.681431e+07,1.234705e+08,1.242662e+08,0.722066,4.764211e+08,2.023307e+07,1.089197e+08
min,0.000000e+00,3.000000e+04,0.000000e+00,1999.000000,3.000000e+04,0.000000e+00,0.000000e+00
25%,0.000000e+00,5.000000e+06,0.000000e+00,2013.000000,6.472500e+06,0.000000e+00,0.000000e+00
50%,0.000000e+00,2.500000e+07,2.000000e+07,2013.000000,3.500000e+07,0.000000e+00,0.000000e+00
75%,3.700000e+07,9.045000e+07,8.625000e+07,2013.000000,1.021250e+08,1.695000e+06,0.000000e+00
max,6.000000e+08,1.307800e+09,1.307800e+09,2015.000000,5.170000e+09,3.650000e+08,1.307800e+09


In [15]:
df[['project_name', 'countryname', 'mjtheme_namecode','mjtheme']].head()

,project_name,countryname,mjtheme_namecode,mjtheme
0,Ethiopia General Education Quality Improvement...,Federal Democratic Republic of Ethiopia,"[{'code': '8', 'name': 'Human development'}, {...",[Human development]
1,TN: DTF Social Protection Reforms Support,Republic of Tunisia,"[{'code': '1', 'name': 'Economic management'},...","[Economic management, Social protection and ri..."
2,Tuvalu Aviation Investment Project - Additiona...,Tuvalu,"[{'code': '5', 'name': 'Trade and integration'...","[Trade and integration, Public sector governan..."
3,Gov't and Civil Society Organization Partnership,Republic of Yemen,"[{'code': '7', 'name': 'Social dev/gender/incl...","[Social dev/gender/inclusion, Social dev/gende..."
4,Second Private Sector Competitiveness and Econ...,Kingdom of Lesotho,"[{'code': '5', 'name': 'Trade and integration'...","[Trade and integration, Financial and private ..."


In [16]:
count1 = df['countryname'].value_counts() 
print(count1)

People's Republic of China                  19
Republic of Indonesia                       19
Socialist Republic of Vietnam               17
Republic of India                           16
Republic of Yemen                           13
Kingdom of Morocco                          12
People's Republic of Bangladesh             12
Nepal                                       12
Republic of Mozambique                      11
Africa                                      11
Federative Republic of Brazil                9
Burkina Faso                                 9
Islamic Republic of Pakistan                 9
United Republic of Tanzania                  8
Republic of Tajikistan                       8
Republic of Armenia                          8
Federal Republic of Nigeria                  7
Hashemite Kingdom of Jordan                  7
Kyrgyz Republic                              7
Lao People's Democratic Republic             7
Democratic Republic of the Congo             6
Republic of K

In [17]:
#Question : 1
#Find the 10 countries with most projects
print(count1.head(10))

People's Republic of China         19
Republic of Indonesia              19
Socialist Republic of Vietnam      17
Republic of India                  16
Republic of Yemen                  13
Kingdom of Morocco                 12
People's Republic of Bangladesh    12
Nepal                              12
Republic of Mozambique             11
Africa                             11
Name: countryname, dtype: int64


In [20]:
print(df['mjtheme_namecode'].head())

0    [{'code': '8', 'name': 'Human development'}, {...
1    [{'code': '1', 'name': 'Economic management'},...
2    [{'code': '5', 'name': 'Trade and integration'...
3    [{'code': '7', 'name': 'Social dev/gender/incl...
4    [{'code': '5', 'name': 'Trade and integration'...
Name: mjtheme_namecode, dtype: object


In [32]:
#Q2
from pandas.io.json import json_normalize
df_themes = json_normalize(data, 'mjtheme_namecode', ['id'])
# some projects seem to have multiple theme code's
#group by now on id, and sort descending order

# so to find top themes we will account for this one to many relationship
print ('Top 10 Major World Bank Project Themes:')
df_themes.name.value_counts().head(10)
# looks like [name] missing for some rows

Top 10 Major World Bank Project Themes:


Environment and natural resources management    223
Rural development                               202
Human development                               197
Public sector governance                        184
Social protection and risk management           158
Financial and private sector development        130
                                                122
Social dev/gender/inclusion                     119
Trade and integration                            72
Urban development                                47
Name: name, dtype: int64

In [34]:
#Q3

# create lookup table from code to name
#HINT; group by on column column
df_themes_name_to_code = df_themes.groupby('add your column').code.max().sort_values(ascending=False)
# drop the missing name rows
df_themes_name_to_code = df_themes_name_to_code[df_themes_name_to_code.index != '']
# convert to df
df_themes_name_to_code = pd.DataFrame(df_themes_name_to_code,columns=['code'])
# pull name into a column
df_themes_name_to_code['name_clean'] = df_themes_name_to_code.index
# set code to be the index
df_themes_code_to_name = df_themes_name_to_code.set_index(['code'])

# now merge on the name based on the code for the missing projects
print df_themes.shape
df_themes_clean = df_themes.merge(df_themes_code_to_name,how='outer',left_on=['code'],right_index=True)
print df_themes_clean.shape

In [36]:
# get a list of cleaned theme names by id 
df_theme_names = pd.DataFrame(df_themes_clean.groupby('id').apply(lambda x: '|'.join(x['name_clean'])),columns=['theme_names'])
# get s list of theme codes by id
df_theme_codes = pd.DataFrame(df_themes.groupby('id').apply(lambda x: '|'.join(x['code'])),columns=['theme_codes'])
# now make an id level lookup table thsat can be used later to merge to original data
df_theme_lut = df_theme_names.merge(df_theme_codes,left_index=True,right_index=True)
# look at shape
print (df_theme_lut.shape)
# take a look
df_theme_lut.head()

(500, 2)


,theme_names,theme_codes
id,,
P075941,Trade and integration|Social dev/gender/inclus...,4|7|10|5
P085621,Environment and natural resources management|E...,11|7|11|11
P086592,Environment and natural resources management|R...,11|10|10|10
P094183,Trade and integration|Trade and integration|Ru...,10|5|5
P095003,Trade and integration|Rural development,5|10


In [38]:
# merge back out cleaned fields into the original data
df_cleaned = df.merge(df_theme_lut,how='right',left_on=['id'],right_index=True,)

# make sure shape has has not changed
print (df.shape)
print (df_cleaned.shape)
# we expect to just see the addition of 2 cols

# take a look at the fields to see the cleaning we have done
df_cleaned[['id','mjtheme_namecode','theme_names','theme_codes']].head(10)

(500, 50)
(500, 52)


,id,mjtheme_namecode,theme_names,theme_codes
0,P129828,"[{'code': '8', 'name': 'Human development'}, {...",Human development|Environment and natural reso...,8|11
1,P144674,"[{'code': '1', 'name': 'Economic management'},...",Economic management|Social protection and risk...,1|6
2,P145310,"[{'code': '5', 'name': 'Trade and integration'...",Environment and natural resources management|S...,5|2|11|6
3,P144665,"[{'code': '7', 'name': 'Social dev/gender/incl...",Social dev/gender/inclusion|Social dev/gender/...,7|7
4,P144933,"[{'code': '5', 'name': 'Trade and integration'...",Trade and integration|Financial and private se...,5|4
5,P146161,"[{'code': '6', 'name': 'Social protection and ...",Social protection and risk management|Social p...,6|6
6,P121185,"[{'code': '2', 'name': 'Public sector governan...",Public sector governance|Financial and private...,2|4
7,P127033,"[{'code': '11', 'name': 'Environment and natur...",Human development|Environment and natural reso...,11|8
8,P130164,"[{'code': '10', 'name': 'Rural development'}, ...",Social dev/gender/inclusion|Rural development,10|7
9,P130903,"[{'code': '2', 'name': 'Public sector governan...",Public sector governance|Public sector governa...,2|2|2
